In [66]:
import pandas as pd
import numpy as np
df=pd.read_csv("C:/Users/STUDENT/Desktop/datasets/KAGGLE/titanic/train.csv")
df.head()
x=df

In [67]:
y=x.pop("Survived")
numeric_variable=list(df.dtypes[df.dtypes!="object"].index)
df[numeric_variable].tail()


,PassengerId,Pclass,Age,SibSp,Parch,Fare
886,887,2,27.0,0,0,13.00
887,888,1,19.0,0,0,30.00
888,889,3,NaN,1,2,23.45
889,890,1,26.0,0,0,30.00
890,891,3,32.0,0,0,7.75


In [68]:
x["Age"].fillna(x.Age.mean(),inplace=True)

In [69]:
x[numeric_variable].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
PassengerId    891 non-null int64
Pclass         891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
dtypes: float64(2), int64(4)
memory usage: 41.8 KB


In [70]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [71]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x[numeric_variable],y,test_size=0.1)

In [72]:
x_train.values

array([[449.    ,   3.    ,   5.    ,   2.    ,   1.    ,  19.2583],
       [758.    ,   2.    ,  18.    ,   0.    ,   0.    ,  11.5   ],
       [538.    ,   1.    ,  30.    ,   0.    ,   0.    , 106.425 ],
       ...,
       [254.    ,   3.    ,  30.    ,   1.    ,   0.    ,  16.1   ],
       [628.    ,   1.    ,  21.    ,   0.    ,   0.    ,  77.9583],
       [492.    ,   3.    ,  21.    ,   0.    ,   0.    ,   7.25  ]])

In [73]:
from keras.utils import np_utils
y_train=y_train.as_matrix()
y_test=y_test.as_matrix()

y_train=np_utils.to_categorical(y_train,2)
y_test=np_utils.to_categorical(y_test,2)

C:\Users\STUDENT\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\STUDENT\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [74]:
y_train[472]

array([1., 0.], dtype=float32)

In [75]:
from keras.models import Sequential
from keras.layers import Activation,Dense

In [76]:
output_dim = 2
input_dim = x_train.shape[1]

batch_size = 100
nb_epoch = 20

In [77]:
from keras.layers import Dropout
from keras.layers import BatchNormalization

model_drop = Sequential()

model_drop.add(Dense(256, activation='relu', input_shape=(input_dim,), kernel_initializer="he_normal"))
model_drop.add(BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(128, activation='relu', kernel_initializer="he_normal") )
model_drop.add(BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(output_dim, activation='softmax'))


model_drop.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 256)               1792      
_________________________________________________________________
batch_normalization_13 (Batc (None, 256)               1024      
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 128)               32896     
_________________________________________________________________
batch_normalization_14 (Batc (None, 128)               512       
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 258       
Total para

In [79]:
model_drop.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
results= model_drop.fit(x_train.values, y_train, batch_size=batch_size, epochs=1000, verbose=0, validation_data=(x_test.values, y_test))

In [80]:
score = model_drop.evaluate(x_test, y_test, verbose=1) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

90/90 [==============================] - 0s 174us/step
Test score: 0.7340052392747667
Test accuracy: 0.6666666746139527


In [86]:
test=pd.read_csv("C:/Users/STUDENT/Desktop/datasets/KAGGLE/titanic/test.csv")
test=test[numeric_variable]
test['Age'] = test['Age'].fillna(test['Age'].median())
test['Fare'] = test['Fare'].fillna(test['Fare'].median())


prediction = model_drop.predict_classes(test.values, batch_size=32)



In [87]:
output = pd.DataFrame({'PassengerId': test.PassengerId,'Survived': prediction})
output.to_csv('submission1.csv', index=False)
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [88]:
test.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
0,892,3,34.5,0,0,7.8292
1,893,3,47.0,1,0,7.0000
2,894,2,62.0,0,0,9.6875
3,895,3,27.0,0,0,8.6625
4,896,3,22.0,1,1,12.2875
